In [19]:
import urllib.request as urlrq
import certifi
import ssl
import re
import pandas as pd
from bs4 import BeautifulSoup
pd.set_option('display.max_columns', 999)

resp = urlrq.urlopen('https://www.wikidex.net/wiki/Lista_de_Pok%C3%A9mon', context=ssl.create_default_context(cafile=certifi.where()))
html_bytes = resp.read()
html = html_bytes.decode("utf-8")
soup = BeautifulSoup(html, "html.parser")
tables=soup.find_all('table',{'class':['tabpokemon', 'sortable' ,'mergetable' ,'jquery-tablesorter']})
i=0
list_pokemon=[]
for table in tables:
    tbody=table.find('tbody')
    trlist=tbody.find_all('tr')
    for tr in trlist:
        tdlist=tr.find_all('td')
        for td in tdlist:
            if(i==1):
                list_pokemon.append(td.get_text().replace('\n','').lower())
            i+=1
        i=0
#print(list_pokemon)




#Get Pokemon Stats
resp_stats = urlrq.urlopen('https://serebii.net/pokedex-swsh/'+list_pokemon[0], context=ssl.create_default_context(cafile=certifi.where()))
html_bytes_stats = resp_stats.read()
html_stats = html_bytes_stats.decode("latin-1")
soup_stats = BeautifulSoup(html_stats, "html.parser")
table_stats = soup_stats.find_all('table',class_='dextable')
i=0
j=0
k=0
pokemon_stats=[]
for table in table_stats:
    if(i==18):
        trlist=table.find_all('tr')
        for tr in trlist:
            if(j==2):
                tdlist=tr.find_all('td')
                for td in tdlist:
                    if(k!=0):
                        pokemon_stats.append(td.string)
                    k+=1
            j+=1
    i+=1
#print(pokemon_stats)
pokemon_stats.insert(0, list_pokemon[0].capitalize())
#print(dataframe)

#Get Pokemon Weaknesses
resp_stats = urlrq.urlopen('https://serebii.net/pokedex-swsh/'+list_pokemon[0], context=ssl.create_default_context(cafile=certifi.where()))
html_bytes_stats = resp_stats.read()
html_stats = html_bytes_stats.decode("latin-1")
soup_stats = BeautifulSoup(html_stats, "html.parser")
table_stats = soup_stats.find_all('table',class_='dextable')
i=0
j=0
k=0
pokemon_weaknesses=[]
aux_weakness=""
for table in table_stats:
    if(i==3):
        trlist=table.find_all('tr')
        for tr in trlist:
            if(j==2):
                tdlist=tr.find_all('td')
                for td in tdlist:
                    aux_weakness=td.string
                    aux_weakness=aux_weakness.replace("*","")
                    aux_float=float(aux_weakness)
                    pokemon_weaknesses.append(aux_float)
            j+=1
    i+=1
#print(pokemon_weaknesses)


#Get Pokemon Name in Other Languages
tableaux = soup_stats.find_all('table',class_='dextable')
i=0
j=0
k=0
l=0
m=0
n=0
o=0
pokemon_languages=[]
for table in tableaux:
    if(i==1):
       
        trlist=table.find_all('tr')
        for tr in trlist:
            if(j==1):
               
                tdlist=tr.find_all('td')
                for td in tdlist:
                    if(k==1):
                        tablelist=td.find_all('table')
                        for table in tablelist:
                            trlist_in_table=table.find_all('tr')
                            for tr in trlist_in_table:
                                if(m==0 or m==1):
                                    tdlist_in_tr=tr.find_all('td')
                                    for td in tdlist_in_tr:
                                        if(n%2==1):
                                            if(len(td.contents)==3):
                                                pokemon_languages.append(str(td.contents[0].string))
                                                pokemon_languages.append(str(td.contents[2].string))
                                            else:
                                                pokemon_languages.append(str(td.contents[0].string))
                                        n+=1
                            m+=1        
                    k+=1
            j+=1
    i+=1


#Get Pokemon Type(s)
resp_stats = urlrq.urlopen('https://serebii.net/pokedex-swsh/'+list_pokemon[0], context=ssl.create_default_context(cafile=certifi.where()))
html_bytes_types = resp_stats.read()
html_types = html_bytes_types.decode("latin-1")
soup_types = BeautifulSoup(html_types, "html.parser")
table_types = soup_types.find_all('table',class_='dextable')
i=0
j=0
k=0
m=0
pokemon_types=[]
aux_type=''
for table in table_types:
    if(i==1):
        trlist=table.find_all('tr')
        for tr in trlist:
            if(j==1):
                tdlist=tr.find_all('td',class_='cen')
                for td in tdlist:
                    alist=td.find_all('a')
                    for a in alist:
                        aux_type=a['href']
                        aux_type=aux_type.replace('/pokedex-swsh/','')
                        aux_type=aux_type.replace('.shtml','')
                        aux_type=aux_type.capitalize()
                        pokemon_types.append(aux_type)
                        m+=1
                    if(m==1):
                        pokemon_types.append(None)
                   
            j+=1
    i+=1
#print(pokemon_types)

pokemon_stats.extend(pokemon_languages)
pokemon_stats.extend(pokemon_types)
pokemon_stats.extend(pokemon_weaknesses)
dataframe=pd.DataFrame([pokemon_stats], columns =['Name','HP', 'Attack', 'Defense','Sp.Attack','Sp.Defense','Speed','JapaneseName','JapaneseNameChar','FrenchName','GermanName','KoreanNamechar',
'Type1','Type2','WeakNormal','WeakFire','WeakWater','WeakElectric','WeakGrass','WeakIce','WeakFight','WeakPoison','WeakGround','WeakFlying','WeakPsychic','WeakBug','WeakRock','WeakGhost','WeakDragon','WeakDark','WeakSteel','WeakFairy'])
print (dataframe)

        Name  HP Attack Defense Sp.Attack Sp.Defense Speed JapaneseName  \
0  Bulbasaur  45     49      49        65         65    45  Fushigidane   

  JapaneseNameChar  FrenchName GermanName KoreanNamechar  Type1   Type2  \
0            フシギダネ  Bulbizarre    Bisasam           이상해씨  Grass  Poison   

   WeakNormal  WeakFire  WeakWater  WeakElectric  WeakGrass  WeakIce  \
0         1.0       2.0        0.5           0.5       0.25      2.0   

   WeakFight  WeakPoison  WeakGround  WeakFlying  WeakPsychic  WeakBug  \
0        0.5         1.0         1.0         2.0          2.0      1.0   

   WeakRock  WeakGhost  WeakDragon  WeakDark  WeakSteel  WeakFairy  
0       1.0        1.0         1.0       1.0        1.0        0.5  
